In [1]:
 from tqdm.auto import tqdm
%pdb on
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
import re
import spacy
from collections import OrderedDict
from functools import partial
import pandas as pd
import numpy as np
import copy
#from sense2vec import Sense2Vec
import torch
try:
    from transformers import AutoTokenizer, AutoModelForCausalLM, OPTForCausalLM,AutoModelForSeq2SeqLM
except:
    !pip install transformers
    from transformers import AutoTokenizer, AutoModelForCausalLM, OPTForCausalLM,AutoModelForSeq2SeqLM
from typing import List, Tuple
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter
import nltk
nltk.download('punkt')
try:
    from sentence_transformers import SentenceTransformer 
except:
    !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

nltk.download('omw-1.4')

/home/ubuntu/miniforge3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Automatic pdb calling has been turned ON


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [55]:

class distractors_by_autoregressive_model:
    def __init__(self,device='cpu',emb_model='all-MiniLM-L6-v2',
                 autoregressive_model_checkpoint = "facebook/galactica-1.3b") -> None:
        #"facebook/galactica-1.3b" "facebook/galactica-6.7b" "EleutherAI/gpt-neo-1.3B"
        self.MODEL_EMBEDDING =  SentenceTransformer(emb_model)

        self.DEVICE = device
        self.USE_ANSWER_WITH_VARYING_SPANS = False
        self.max_letters_in_word = 20 # more than `self.max_letters_in_word` letters in word suspected to be non-word sequence. 

        '''
        checkpoint = "EleutherAI/gpt-neo-1.3B" #"gpt2" EleutherAI/gpt-neo-1.3B EleutherAI/gpt-neo-2.7B
        MODEL = AutoModelForCausalLM.from_pretrained(checkpoint).to(DEVICE)
        TOKENIZER = AutoTokenizer.from_pretrained(checkpoint)
        '''
        #autoregressive_model_checkpoint = "google/flan-t5-xl"
        #self.model = AutoModelForSeq2SeqLM.from_pretrained(autoregressive_model_checkpoint).to(device)
        

        self.model =  OPTForCausalLM.from_pretrained(autoregressive_model_checkpoint).to(device)
        self.tokenizer =  AutoTokenizer.from_pretrained(autoregressive_model_checkpoint)

        lemmatizer = WordNetLemmatizer()
        self.lemma_get = lemmatizer.lemmatize
        torch.manual_seed(0)

    def get_answer_and_distractor_embeddings(self,answer,candidate_distractors):
        answer_embedding = self.MODEL_EMBEDDING.encode([answer])
        distractor_embeddings = self.MODEL_EMBEDDING.encode(candidate_distractors)
        return answer_embedding, distractor_embeddings

    def remove_prefix(self,text, prefix):
        print('text',text)
        print('prefix',prefix)
        return text[len(prefix):].strip()
        #return text.replace(prefix.strip(),'').strip()


    def only_alphanumeric_punctuation(self,text):
        match = re.search("^[\w\s\d\.,%=+:\(\)-]+$", text)
        return bool(match)
    
    def too_long_word(self,text):
        #print([len(word) for word in text.split()])
        match = [len(word) > self.max_letters_in_word for word in text.split()]
        return any(match)
    
    def filter_outputs(self,outputs, prefix, answer):
        outputs_filtered = []
        answer_len = len(answer.split())

        for output in outputs:

            output = self.remove_prefix(output, prefix)

            #if "\nAuthors" in output: # for Galactica model
            #    output = output[:output.find('\nAuthors')]
            output = output.replace("e.g.","eg").replace("i.e.","ie").replace("i. e.","ie").replace("e. g.","eg")
            
            # if "." not in output:
            #     continue
            if "." in output:
        
                output_split = output.split(". ")  # consider take all sentences until the last dot. maybe depend on the answers length.

                sentence_len = [len(out.split()) for out in output_split]
                max_sentence_taken_idx = np.argmin(np.abs(np.cumsum(sentence_len)-answer_len))        
                output_cut = ". ".join(output_split[:max_sentence_taken_idx+1])+"."

            else:
                output_cut = output
            print('3 output_cut',output_cut)
            #print(output_cut)
            # if not self.only_alphanumeric_punctuation(output_cut):    
            #     continue
            if self.too_long_word(output_cut):
                print('too long word')
                continue
            #if len(self.tokenizer.encode(output_cut)) > 2:
            outputs_filtered.append(output_cut.strip().replace("\n"," "))

        return outputs_filtered

    def generate_distractors(self, question, answer,title=False,prefix=False):
        if self.USE_ANSWER_WITH_VARYING_SPANS:
            if len(answer.split()) <= 3:
                num_words_to_add = (1,2)
            elif len(answer.split()) <= 5:
                num_words_to_add = (1,3)
            elif len(answer.split()) <= 7:
                num_words_to_add = (2,4)
            elif len(answer.split()) <= 10:
                num_words_to_add = (1,3,5)
            else: 
                num_words_to_add = (2,4,6)
            num_return_sequences = 10
        else:

            punct_or_stopwords_or_in_question = set([*stopwords.words('english'), *string.punctuation, *[self.lemma_get(qe.lower()) for qe in question.replace('?','').split()]])

            num_words_to_add=[1]
            for word in answer.split()[1:]:
                if self.lemma_get(word.lower()) in punct_or_stopwords_or_in_question:
                    num_words_to_add[0]+=1
                else:
                    break
            if " ".join(answer.split()[:num_words_to_add[0]]).strip().lower() in ['a','an','the']:
                num_words_to_add[0]+=1
            if len(answer.split())==num_words_to_add[0]:
                # if all answer was selected as the prefix or by :
                #       (1) The answer is only one word 
                #       (2) All the words from the question are in the answer.
                #       The distractors cannot be made.
                return False 
            num_return_sequences = 25
        candidate_distractors = []
        if not prefix:
            if title:
                prefix = title +". "+question +" " # "Title: "+title+". "+question + " "
                #prefix = "Title:"+ title +". Answer to the question: "+question  # "Title: "+title+". "+question + " "
                #prefix = "Title:"+ title +". </s> Answer to the question: "+question+" </s> "  # "Title: "+title+". "+question + " "
                #prefix = "Answer to the question: "+question+" /n/n Context: "+ title +". /n/n "  # "Title: "+title+". "+question + " "
                #prefix = "Answer to the question: "+question+" </s> Context: "+ title +". </s> Answer: "  # "Title: "+title+". "+question + " "
                #prefix = f"Context: {title}\n\nQuestion: {question}\n\nAnswer:"
            else:
                prefix = "Answer to the question: " + question 
            '''
            if title:
                prefix = "Answer to the question: "+question + " Context: "+title +" " # "Title: "+title+". "+question + " "
            else:
                prefix = "Answer to the question: " + question + " "
            '''
        if isinstance(prefix,list):
            prefix_list = prefix
        else:
            prefix_list = [prefix]
        #when `self.USE_ANSWER_WITH_VARYING_SPANS` is False, there is only one element in `num_words_to_add` so the for is with one iteration
        for num in num_words_to_add:
            for prefix in prefix_list:
                prompt =  prefix + " ".join(answer.split()[:num])+" "
                print(prompt)
                input_ids = self.tokenizer.encode(prompt, return_tensors='pt').to(self.model.device)
                
                # output_ids = self.model.generate(
                #     input_ids, 
                #     do_sample = True, 
                #     min_length = len(self.tokenizer.encode(prompt)) + 5,
                #     max_length = len(self.tokenizer.encode(prompt)) + 25+25,
                #     top_p = 0.94, # 0.8 
                #     top_k = 30,   #30
                #     repetition_penalty  = 2.0, # 10.0,
                #     temperature = 2.0,
                #     num_return_sequences = num_return_sequences,
                #     #early_stopping= True
                # ).cpu()
                
                output_ids = self.model.generate(input_ids,max_new_tokens=100,repetition_penalty  = 10.0, diversity_penalty=float(10), # top_k = 20,do_sample=True,
                               num_beams=5,num_return_sequences=5,num_beam_groups=5)
                outputs = [self.tokenizer.decode(output, skip_special_tokens=True) for output in output_ids]
                outputs = [ dist[:dist.find('\n\n')] 
                           if dist.find('\n\n') else dist
                           for dist in outputs ]
                print(len(outputs),outputs) 
                candidate_distractors += self.filter_outputs(outputs, prefix,answer)
        if len(candidate_distractors)==0:
            print('no candidate_distractors pass filtring')
            return False
        answer_embedding, distractor_embeddings = self.get_answer_and_distractor_embeddings(answer, candidate_distractors)
        #distractors = self.mmr(answer_embedding, distractor_embeddings, candidate_distractors, reverse=True, top_n=5, diversity=0.8)
        distractors = candidate_distractors
        if distractors: # remove multipile spaces
            distractors = [' '.join(d.split()) for d in distractors]
        print('+'*30)
        print(f'{question} ----- {answer}')
        for i,d in enumerate(distractors):
            print(f'({i})-{d}\n')
        return distractors

    def mmr(self,
        doc_embedding: np.ndarray,
        word_embeddings: np.ndarray,
        words: List[str],
        top_n: int = 5,
        diversity: float = 0.8,
        reverse: bool = False
    ) -> List[Tuple[str, float]]:

        keywords_idx = [np.argmin(cosine_similarity(word_embeddings, doc_embedding))]
        doc_embedding = word_embeddings[keywords_idx[0]].reshape(1, -1)
        # word_embeddings = [emb for idx,emb in enumerate(word_embeddings) if idx not in keywords_idx]
        word_similarity = cosine_similarity(word_embeddings)
        word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
        

        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
        # print(words[keywords_idx[0]])

        for _ in range(min(top_n - 1, len(words) - 2)):
            # Extract similarities within candidates and
            # between candidates and selected keywords/phrases
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            target_similarities = np.max(
                word_similarity[candidates_idx][:, keywords_idx], axis=1
            )

            # Calculate MMR
            mmr = (1 - diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
            mmr_idx = candidates_idx[np.argmax(mmr)]

            # Update keywords & candidates
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)

        # Extract and sort keywords in descending similarity
        keywords = [
            (words[idx], round(float(word_doc_similarity.reshape(1, -1)[0][idx]), 4))
            for idx in keywords_idx
        ]
        keywords = sorted(keywords, key=itemgetter(1), reverse=True)
        return [kw[0] for kw in keywords]


In [56]:
if 'di' in locals():
    del di
di = distractors_by_autoregressive_model('cuda')
title ="The role of semantics in the success of crowdfunding projects"
qs1 = "What do the authors analyze?" 
answer1 = "A large dataset of crowdfunding project data"

qs2 ="On what dataset was the proposed model evaluated?"
answer2 = "The proposed models were evaluated on the following datasets: All_D."

qs3 = "What should the post of a project contain?"
answer3 = "more feelings words"

qs4 = "What did the authors use Beautifulsup to gain?"
answer4 = "additional metadata features"

qs5 = "What categories of buzzwords were used in the study?"
answer5 = 'General conversation, education, business, sales and marketing, science and technology, politics and current affairs. The buzzword dataset used in this study contains words from different categories: general conversation, business.'
for qs,answer in zip([qs1,qs2,qs3,qs4,qs5],[answer1,answer2,answer3,answer4,answer5]):
    print()
    pp= di.generate_distractors( qs, answer,title=title)
    print(qs,'---',answer)
    print(pp)


The role of semantics in the success of crowdfunding projects. What do the authors analyze? A large 


/home/ubuntu/miniforge3/lib/python3.9/site-packages/transformers/generation/beam_search.py:198: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


5 ['The role of semantics in the success of crowdfunding projects. What do the authors analyze? A large (n=1,034) dataset was collected from a leading online crowdfunding platform and analyzed to understand how semantic features affect project success. The results show that there is no significant difference between successful and unsuccessful projects with respect to their average number of words per sentence or vocabulary size. However, they found that more than half of the top-5 most frequent words are related to fundraising activities such as “raise”, “fund”, “support”, etc.', 'The role of semantics in the success of crowdfunding projects. What do the authors analyze? A large 12-month dataset from one major online fundraising platform, Kickstarter was analyzed to understand how semantic features affect project outcomes. The results show that a combination of textual and non-textual information can be used to predict whether or not a given funding opportunity will succeed.\n* How do

In [6]:
#("{context}\n\nGenerate a question about the above context.", "{question}\n{options_}"),
# ("Write a question about the following article." "\n\n{context}\n\nQuestion:", "{question}\n{options_}"),
device='cuda'
autoregressive_model_checkpoint = "google/flan-t5-xl"
if not 'model' in locals():
    model = AutoModelForSeq2SeqLM.from_pretrained(autoregressive_model_checkpoint).to(device)
    tokenizer =  AutoTokenizer.from_pretrained(autoregressive_model_checkpoint)
BEAMS = 4
ARGS_GENERATOR = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": BEAMS, #20
    "length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
    'top_p' :0.9,
    #'do_sample':True,
    'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therefore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
    'num_beam_groups':BEAMS, #20 
    "return_dict_in_generate" :True,
    'output_scores':True,
    #"early_stopping": True, 
    'num_return_sequences':3
}
def get_output( text, **cfg):
        input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)
        res = model.generate(input_ids, **cfg) #.cpu()
        r = tokenizer.batch_decode(res['sequences'], skip_special_tokens=True)
        try:    torch.cuda.empty_cache()
        except: pass
        return [r, res['sequences_scores']]
def get_outputs(texts, **cfg):
            return [
                get_output(text, **dict(cfg))
                for text in texts
            ]
sections = ['Crowdfunding platforms allow entrepreneurs to publish projects and raise funds for realizing them. Hence, the question of what influences projects’ fundraising success is very important. Previous studies examined various factors such as project goals and project duration that may influence the outcomes of fundraising campaigns. We present a novel model for predicting the success of crowdfunding projects in meeting their funding goals. Our model focuses on semantic features only, whose performance is comparable to that of previous models. In an additional model we developed, we examine both project metadata and project semantics, delivering a comprehensive study of factors influencing crowdfunding success. Further, we analyze a large dataset of crowdfunding project data, larger than reported in the art. Finally, we show that when combining semantics and metadata, we arrive at F1 score accuracy of 96.2%. We compare our model’s accuracy to the accuracy of previous research models by applying their methods on our dataset, and demonstrate higher accuracy of our model. In addition to our scientific contribution, we provide practical recommendations that may increase project funding success chances.',
 ' In recent years, crowdfunding has emerged as a popular financing mechanism that allows various types of projects to get funded. Three main parties are involved in crowdfunding: entrepreneurs (and their projects), individuals or groups who support the project, and online platforms, i.e., crowdfunding websites. The average success rate of funded crowdfunding projects in Kickstarter is 37.5% [4] In this paper, we refer to a project as a funding success (FS) if it achieved its funding goal (i.e.  In contrast to prior studies, we take a more comprehensive approach to studying and predicting funding success. We use unique features (for example, the use of buzzwords) and perform extensive research on the semantics of the projects’ text. Our main research objective is to find characteristics, with a focus on semantic characteristics, which affect the success of a project in meeting its funding goal via crowdfunding. This approach leverages known results and improves upon them to provide high-quality funding success prediction.  Achieving these research objectives should provide new insights into the relationship between crowdfunding project’s data and their funding success. This should provide entrepreneurs with additional and desirable ways to improve their chances of reaching their funding goals. The accuracy of the semantic-model (in terms of F-score and correctly classified instances) is comparable to the accuracy of state-of-the-art models which are based on metadata features such as the number of updates, number of images, etc.  The accuracy of the combined-model is higher than the accuracy derived in previous studies that use other models [11–14] Fourth, we show that buzzwords and LIWC are among the highly correlate features with the project’s success in fund raising. Fifth, to the best of our knowledge, our research relies on the largest dataset used to date for developing models that predict funding success of crowdfunding projects. In addition to the scientific contributions listed above, we provide a set of recommendations that may increase project funding success chances.  The Methodology section focuses on the research methodology applied, including the acquisition of our dataset, preprocessing, feature extraction and selection. The Results section presents the empirical evaluation of our research model and discusses the results. It further compares our models to models in earlier studies.',
 ' In this section, we discuss the features used in our models, highlighting where we differ from previous studies. We also present the metric used to measure the accuracy of the models. For semantic analysis, the projects’ textual data is required.',
 ' The relationship between funding success and buzzwords used in the description of the project was not examined. The buzzword dataset used in this study contains words from different categories: general conversation, education, business, sales and marketing, science and technology, politics, and current affairs.',
 ' We used the Linguistic Inquiry and Word Count (LIWC) software tool to extract features from the text. LIWC analysis measures the appearance of dictionary words in a specific text. The measure is a numeric value in the range [0..1]. It reflects the degree to which the text being analyzed is related to the theme of the dictionary. The output of LIWC is VD = SD/NT, which is the value of the feature that corresponds to D. Previous studies on crowdfunding used LIWC to analyze the textual description part of projects.  The number and diversity of features in our study are much larger than previously published, and our dataset is significantly greater as well. As a result, we arrive at much higher model accuracy.',
 ' Latent Dirichlet allocation (LDA) is a widely used topic modeling method. LDA algorithm considers each document as a collection of topics, where each word in the document belongs to one or some of these topics. Previous studies on crowdfunding used LDA to perform topic analysis on the text of project updates. We used the Gensim Python package LDA implementation (from GitHub.com) to execute LDA. The latter is known to run faster than other implementations and generates better topic segregation [16].  The three main inputs to the LDA topic model are the dictionary, the corpus, and the number of topics. Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is [16] To avoid overfitting, we chose the one that derived the highest coherence value and the smallest number of. topics. There were 30 topics that derived. the highest. coherence values for the All_D dataset, 15 for the Tech_D and 15 for Market_D datasets.',
 ' We incorporated metadata features known to affect funding success. The metadata features we used were extracted from projects’ posts via Python web scraping. The set we used for our analysis included the number of photos, the. number of videos, the number. of updates, the. number of previously created projects by.',
 ' The following additional topics were deduced: Explanation words: example, explain, for instance, i.e., mean, in other words, in that, that is, etc. Feelings words: angry, annoyed, afraid, awkward, affectionate, anxious, alarmed, awed, aggravated, amazed, amazed.',
 ' We used the following metrics to measure the performance of our models. Recall is the ratio of correctly predicted positive records to the all records in actual class. F-score (also denoted F1) is the weighted harmonic mean of a model’s precision and recall.',
 ' We used a dataset of 50,000 Kickstarter and 50,00 Indiegogo projects. We obtained these data from the Kaggle website for 2018. We used Beautifulsup (a Python web scraping package) to gain additional metadata features, such as, description content, number of updates, etc. For each record in the dataset, we obtained the values of five metadata features and about 120 semantic features, including buzzwords, LIWC outputs, feelings words, explanation words, and LDA outputs. We removed LDA topics that overlapped with LIWC topics by comparing the topic sets.  To study the relationship between features and the project category, we built three datasets: All_D, Tech_D and Market_D. The division aims to address the following analysis goals: First, it facilitates improvement in the LDA topic coherence score. Second, it allows us to determine whether a category affects the features that are highly correlated with funding success. Third, to study the correlation between buzzwords and FS, it is preferable for datasets to be separated into specific subdomains.  For each dataset, the original features space was reduced into three Principal Components (PCs) The sum of the explained variance ratio of these three PCs is 78% of the total variance. To identify the most significant set of features (MSSF) that has the highest impact on FS, we use the CFS (correlation-based feature selection) algorithm. The PCA plots in Fig 1 present the distribution of the All_D, Market_D and Tech_D datasets. Dots in blue represent unsuccessfully funded projects and red represent successfully funded projects.  The models’ design, derivation, and operational tests were conducted using the following Python packages: scikit-learn, scikits-feature and LightGBM. We employed a 10-fold cross-validation test to evaluate the prediction performance. Fig 2 includes a flowchart of the methodology. To validate the results of our study, we compared our model’s accuracy to previous research models by applying their methods on our dataset. To this end, we have developed the combined-model, whose novelty lies in the combination of semantic and meta-data features.',
 'Fig 2. Methodology flowchart.',
 ' In this section, we provide details of the data setup, the usage of the LDA algorithm, the feature correlation analysis, and the feature selection process. In what follows we describe the way in which we found the features that are most influential for FS. The CFS algorithm evaluates subsets of features based on the individual predictive ability of each feature along with the degree of redundancy between them.  The semantic features buzzwords and LIWC are among the features that have a higher correlation in all datasets. An important, unique conclusion of our study is that the features correlated to FS are dependent on the project category. Among the top 10 features, about 70% of the features are the same across datasets. For example, the features feelings_num and buzz_num appear in the three top 10 lists. In contrast, the parameter WC is unique in the R_Tech _D top 10 list and the parameter number is distinctive.',
 ' For the development of the Semantic-Model, semantic features were used as input. We utilized several machine learning algorithms (including SVM, J48, Random Forest, LightGBM, SDG, DNN, and more) on All_D. Table 2 presents the accuracies metrics of the combined-model: Combined model.  The semantic-model we have developed is comparable in accuracy with the state of the art (metadata) models, exhibiting an accuracy level of 91.2%. The LightGBM algorithm exhibited a high accuracy level for all three datasets. In particular, the accuracy of the prediction model is greater than 94% for all datasets.',
 ' The study aims to examine whether the set of features we use for prediction and the dataset on which learning was applied deliver a better model by means of F-score accuracy. We trained the LDA-Model and the Metadata-Model with the algorithms that were used in the studies above, and with additional algorithms, including SVM, J48, Random forest, LightGBM, SDG, and DNN. This training was performed on All_D with 10-fold cross-validation.  F-score is used for consistency with the earlier studies to which we compare. Figure shows that the model we developed has the highest accuracy. The accuracy of the Semantic-Model is similar to that of the LDA-Model and the Metadata-Model.',
 ' The study is the first that investigates the relationship between funding success and buzzwords. The buzzwords feature is among the features that are highly correlated to funding success compared to other parameters that we examined and that other researchers examined. We developed a novel model based on semantic features only and achieved similar accuracy level as previous studies. We also developed a prediction model with an impressive Fscore of 96.2%, focusing on both project-specific aspects and semantics of project descriptions. The results of our study are highly relevant to fundraisers using crowdfunding web platforms.  Future research could further improve accuracy by considering the characteristics of images, video content and the semantics of video scripts. Further improvements in the model’s performance could be achieved via novel feature selection algorithms.',
]
get_outputs(
            [
                f"Write a question about the following article\n\n{cur[:trancuted]}\n\nQuestion:"
                for cur in sections
            ],
            **ARGS_GENERATOR
        )

[[['Why are crowdfunding platforms important?',
   'What is the main purpose of crowdfunding platforms?',
   'Why are crowdfunding platforms important? -'],
  tensor([-17.5531, -24.5470, -51.0471], device='cuda:0')],
 [['How many parties are involved in crowdfunding?',
   'What is the average success rate of funded crowdfunding projects in Kickstarter?',
   'Who are involved in crowdfunding?'],
  tensor([ -7.8947, -22.9828, -24.6290], device='cuda:0')],
 [['Why are we different from previous studies? Available choices: (i). We use different features. (ii). None of the above choices. (III). We have different metric. (IV). We are using different models.',
   'What is the metric used to measure the accuracy of the models? Pick from: (i). The metric is used to determine the accuracy. (ii). None of the above choices. (III). The model is used for semantic analysis. (IV). The models are used for textual analysis.',
   'Why are we different from previous studies? Available choices: (i). We use